In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import datetime 
import time 


In [2]:
pttUrl = "https://www.ptt.cc"
fundUrl = pttUrl +"/bbs/Fund"

In [3]:
r = requests.get(fundUrl) #將此頁面的HTML GET下來
print(r.text) #印出HTML

<!DOCTYPE html>
<html>
	<head>
		<meta charset="utf-8">
		

<meta name="viewport" content="width=device-width, initial-scale=1">

<title>看板 Fund 文章列表 - 批踢踢實業坊</title>

<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.27/bbs-common.css">
<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.27/bbs-base.css" media="screen">
<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.27/bbs-custom.css">
<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.27/pushstream.css" media="screen">
<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.27/bbs-print.css" media="print">




	</head>
    <body>
		
<div id="topbar-container">
	<div id="topbar" class="bbs-content">
		<a id="logo" href="/bbs/">批踢踢實業坊</a>
		<span>&rsaquo;</span>
		<a class="board" href="/bbs/Fund/index.html"><span class="board-label">看板 </span>Fund</a>
		<a class="right small" href="/about.html">關於我們</a>
		<a class="right small" href="/contact.html">聯絡資

## 抓出最後一頁

In [4]:
soup = BeautifulSoup(r.text,"lxml") #將網頁資料以lxml
s = soup.findAll("a") 
p = []
for s in s:
    href = str(s.get('href'))
    matchObj = re.search('index[0-9]{2,}\\.html',href)
    if  matchObj != None :
        p.append(href)


In [5]:
lastPageUrl = pttUrl + str(p[0])
lastPageUrl

'https://www.ptt.cc/bbs/Fund/index1088.html'

In [6]:
latestPage = int(re.findall('[0-9]+', str(p))[0])
latestPage

1088

## 從最後一頁開始回推 

In [7]:
linksArticle = []
pageLength = 1 #要抓幾頁
for pageIndex in range((latestPage-pageLength),latestPage+1):
    link = str(fundUrl) + "/index" + str(pageIndex) + ".html"
    print(link)
    r = requests.get(link) #將網頁資料GET下來
    soup = BeautifulSoup(r.text,"lxml") #將網頁資料以lxml
    sel = soup.select("div.title a") #取HTML標中的 <div class="title"></div> 中的<a>標籤存入sel
    links = [sel.get("href") for sel in sel]
    links = list(links)
    linksArticle = linksArticle + links

https://www.ptt.cc/bbs/Fund/index1087.html
https://www.ptt.cc/bbs/Fund/index1088.html


In [8]:
len(linksArticle)#文章數

40

In [9]:
linksArticle = list(dict.fromkeys(linksArticle))#刪除重複

In [10]:
len(linksArticle)

40

## 抓取文章與推文

In [11]:
articleTable = pd.DataFrame() # 建⽴⽂章 儲存空間
articlePushTable= pd.DataFrame() # 建⽴每篇文章推⽂儲存空間
pushTable = pd.DataFrame() # 建⽴推⽂儲存空間

In [12]:
for tempLink in linksArticle:
    articleUrl = pttUrl + tempLink # ⽂章網址
    r = requests.get(articleUrl) #將網頁資料GET下來
    soup = BeautifulSoup(r.text,"lxml") #將網頁資料以lxml
    
    #內文
    sel = soup.select("span.article-meta-value") #取內文
    author = sel[0].text.split(' ')[0]
    category = sel[1].text
    title = sel[2].text
    dateTime = sel[3].text
    content = soup.find(id="main-content").text
    target_content = u'※ 發信站: 批踢踢實業坊(ptt.cc),'
    #去除掉 target_content
    content = content.split(target_content)
    #去除掉 datetime
    content = content[0].split(dateTime)
    #去除掉文末 --
    main_content = content[1].replace('--', '').replace("\n","").replace("\\","\\\\").replace('"',r'\"')
    tmp = pd.DataFrame({
        'datetime' : [dateTime],
        'title' : [title],
        'author' : [author],
        'content' : [main_content],
        'url' : [articleUrl]
    })
    articleTable = articleTable.append(tmp)
    
    #推文
    for tag in soup.select('div.push'):
        # pushTag  推文標籤  推  噓  註解(→)
        pushTag = tag.find("span", {'class': 'push-tag'}).text
        # print "push_tag:",push_tag

        # pushUserid 推文使用者id
        pushUserid = tag.find("span", {'class': 'push-userid'}).text
        # print "push_userid:",push_userid

        # pushContent 推文內容
        pushContent = tag.find("span", {'class': 'push-content'}).text
        pushContent = pushContent[1:]
        # print "push_content:",push_content

        # push-ipdatetime 推文時間
        pushIpDatetime = tag.find("span", {'class': 'push-ipdatetime'}).text
        pushIpDatetime = pushIpDatetime.rstrip().replace('/','-')
        
        tmp = pd.DataFrame({
            'tag' : [pushTag],
            'userid' : [pushUserid],
            'content' : [pushContent],
            'datetime' : [pushIpDatetime],
            'url' : [articleUrl]
        })
        articlePushTable = articlePushTable.append(tmp)
    pushTable = pushTable.append(articlePushTable)

articleTable.reset_index(drop=True, inplace=True) # index reset
pushTable.reset_index(drop=True, inplace=True) # index reset

## 文章

In [13]:
articleTable.head()

,datetime,title,author,content,url
0,Tue Feb 8 16:09:21 2022,[情報] 元宇宙線上論壇講座,Horacio,元宇宙線上論壇想要了解元宇宙的未來商機嗎加入摩根LINE官方帳號或是掃QR CODE可以獲得...,https://www.ptt.cc/bbs/Fund/M.1644307763.A.D50...
1,Tue Feb 8 22:30:06 2022,[問題] 鉅亨網定期定額扣款日異動,NYYLG,在鉅亨網想改某基金定期定額的日期可是到異動輸入那步驟時按下一步一直無法到異動確認狀態是顯示異...,https://www.ptt.cc/bbs/Fund/M.1644330608.A.392...
2,Wed Feb 9 12:01:26 2022,[新聞] 基富通去年業績再締新猷，客戶數、資產規,rabbit542254,1.原文連結：https://bit.ly/3HzCQtZ2.內容：基富通去年業績再締新猷，...,https://www.ptt.cc/bbs/Fund/M.1644379288.A.039...
3,Wed Feb 9 14:04:48 2022,[問題] 前天基金只扣一次......扼腕不已！！,charles0939,原來開春2/7會扣二次但我只手動放扣一次的金額結果這三天台股大漲扼腕阿！！請問過年九天有碰到...,https://www.ptt.cc/bbs/Fund/M.1644386690.A.2F4...
4,Wed Feb 9 20:29:01 2022,[新聞] 上海私募基金經理高杉淪黃浦江水流屍,nisi0773,原標題:失蹤三周 上海私募基金經理高杉淪黃浦江水流屍1.原文連結：https://www.c...,https://www.ptt.cc/bbs/Fund/M.1644409745.A.CD4...


## 推文

In [14]:
pushTable.head()

,tag,userid,content,datetime,url
0,推,jkinkin,感謝分享,02-08 19:48,https://www.ptt.cc/bbs/Fund/M.1644307763.A.D50...
1,推,jkinkin,感謝分享,02-08 19:48,https://www.ptt.cc/bbs/Fund/M.1644307763.A.D50...
2,→,ilutc,隔一個營業日就能申請異動成功了 別擔心,02-08 22:55,https://www.ptt.cc/bbs/Fund/M.1644330608.A.392...
3,→,jimmyfk,"定期定額是n+1, 14:00才會生效.",02-08 23:09,https://www.ptt.cc/bbs/Fund/M.1644330608.A.392...
4,噓,pk0943,當這裡客服?,02-12 22:58,https://www.ptt.cc/bbs/Fund/M.1644330608.A.392...


In [15]:
linksArticle = list(dict.fromkeys(pushTable['url']))#刪除重複
len(linksArticle)

38

In [16]:
articleTable.dropna().shape

(40, 5)

In [17]:
pushTable.dropna().shape

(14329, 5)

## 存成csv

In [19]:
articleTable.to_csv("pttfund_utf8.csv", index=False, encoding='utf_8_sig')

## 試著讀取，確認是否存成功

In [20]:
csv_name="pttfund_utf8.csv"

In [21]:
data = pd.read_csv(csv_name)
data = data.dropna(subset=['content'])
data

,datetime,title,author,content,url
0,Tue Feb 8 16:09:21 2022,[情報] 元宇宙線上論壇講座,Horacio,元宇宙線上論壇想要了解元宇宙的未來商機嗎加入摩根LINE官方帳號或是掃QR CODE可以獲得...,https://www.ptt.cc/bbs/Fund/M.1644307763.A.D50...
1,Tue Feb 8 22:30:06 2022,[問題] 鉅亨網定期定額扣款日異動,NYYLG,在鉅亨網想改某基金定期定額的日期可是到異動輸入那步驟時按下一步一直無法到異動確認狀態是顯示異...,https://www.ptt.cc/bbs/Fund/M.1644330608.A.392...
2,Wed Feb 9 12:01:26 2022,[新聞] 基富通去年業績再締新猷，客戶數、資產規,rabbit542254,1.原文連結：https://bit.ly/3HzCQtZ2.內容：基富通去年業績再締新猷，...,https://www.ptt.cc/bbs/Fund/M.1644379288.A.039...
3,Wed Feb 9 14:04:48 2022,[問題] 前天基金只扣一次......扼腕不已！！,charles0939,原來開春2/7會扣二次但我只手動放扣一次的金額結果這三天台股大漲扼腕阿！！請問過年九天有碰到...,https://www.ptt.cc/bbs/Fund/M.1644386690.A.2F4...
4,Wed Feb 9 20:29:01 2022,[新聞] 上海私募基金經理高杉淪黃浦江水流屍,nisi0773,原標題:失蹤三周 上海私募基金經理高杉淪黃浦江水流屍1.原文連結：https://www.c...,https://www.ptt.cc/bbs/Fund/M.1644409745.A.CD4...
5,Fri Feb 11 11:32:25 2022,[問題] 群益母子基金請益,wxwlm,前幾天看到youtuber介紹他們家的機制，滿感興趣的，查了一些資料後有一些問題想請教這種母...,https://www.ptt.cc/bbs/Fund/M.1644550347.A.B9C...
6,Sun Feb 13 19:32:01 2022,[閒聊] 今年以來國內外基金表現,uyulala,[閒聊] 今年以來國內外基金表現資料來源https://www.moneydj.com/fu...,https://www.ptt.cc/bbs/Fund/M.1644751923.A.9E7...
7,Mon Feb 14 21:32:25 2022,[新聞] 俄烏關係緊張 投顧:戰爭與經濟利益不一致,nisi0773,1.原文連結：https://www.wealth.com.tw/articles/e31d...,https://www.ptt.cc/bbs/Fund/M.1644845549.A.E87...
8,Wed Feb 16 22:40:49 2022,[問題] 關於美國平衡型基金單筆投入,strafrecht,想請教各位投資達人，因為小弟近期有一點資金，想要購買主要投資標的為美國的平衡型基金，考慮到F...,https://www.ptt.cc/bbs/Fund/M.1645022451.A.FA7...
9,Wed Feb 16 22:48:08 2022,[問題] 資金有限，這兩檔要怎麼分配？,jaekiki,觀察了一陣子國內外基金的表現從去年開始美股就開始一直不起色，今年又要升息變動因素太大覺得定期...,https://www.ptt.cc/bbs/Fund/M.1645022890.A.205...
